# Function Calling / Tool Calling

In [1]:
from openai import AzureOpenAI
client = AzureOpenAI(api_version="2024-12-01-preview")

In [3]:
message = [{'role':'user','content':'what is Quantum Computing.'}]

response = client.chat.completions.create(model='telcogpt',
                                          messages =message,temperature=0.1)

print(response.to_json(indent=2))

{
  "id": "chatcmpl-BWe5btzp1EPSPXPmMPPY9x7bSHnIz",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Quantum computing is a type of computation that takes advantage of the principles of quantum mechanics to process information in fundamentally different ways than classical computers. Here are some key concepts to understand:\n\n1. **Quantum Bits (Qubits)**: Unlike classical bits, which can be either 0 or 1, qubits can exist in a state of superposition, meaning they can represent both 0 and 1 simultaneously. This property allows quantum computers to process a vast amount of information at once.\n\n2. **Superposition**: This principle allows qubits to be in multiple states at the same time. When a quantum computer performs calculations, it can explore many possible solutions simultaneously, potentially leading to faster problem-solving for certain tasks.\n\n3. **Entanglement**: Qubits can become entangled

In [12]:
import requests, json, time

def get_current_weather(city:str)->dict:
    """ this funciton can be used to get current weather information"""
    api_key="6a8b0ac166a37e2b7a38e64416b3c3fe"
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = json.loads(response.content.decode())
    output = {"city":city,"weather":response['weather'][0]['description'],
              "temperature":response['main']['temp'], "unit":"kelvin"
              }
    return output

In [28]:
import wikipedia

def get_wikipedia_summary(query:str)->str:
    response = wikipedia.summary(query)
    return response

In [29]:
get_wikipedia_summary("capital of India")

'This is a list of locations which have served as capital cities in India. The current capital city is New Delhi, which replaced Calcutta in 1911.\n\n'

In [13]:
get_current_weather("delhi")

{'city': 'delhi',
 'weather': 'clear sky',
 'temperature': 312.12,
 'unit': 'kelvin'}

In [ ]:
# 3 components : name of function, description, parameters (desc of params)
tools = [{
    "type":"function",
    "function":{
        "name":"get_current_weather_info",
        "description":"This function can be used to get current weather information for any given city.",
        "parameters":{
            "type":"object",
            "properties":{"city":{"type":"string","description":"name of location/city e.g. delhi, mumbai"}}
        },
        "required":['city',]

    }
},
{
    "type":"function",
    "function":{
        "name":"get_wikipedia_summary",
        "description":"This function can be used to get ONLY information about places, locations, people, historical things for any given query.",
        "parameters":{
            "type":"object",
            "properties":{"query":{"type":"string","description":" query to search on wikipedia e.g. year for second world war"}}
        },
        "required":['query',]

    }
},
]

In [34]:
tool_map = {"get_current_weather_info":get_current_weather,
            "get_wikipedia_summary":get_wikipedia_summary}

def generate_response(prompt):

    messages=[{"role":"system","content":"you are a helpful assistant you provide short & concise answers"},
              {"role":"user",'content':prompt}]
    
    while True:
    
        first_response = client.chat.completions.create(model="telcogpt2",
                                                        messages=messages,temperature=0.2,
                                                        tools=tools,tool_choice='auto')
        if first_response.choices[0].message.tool_calls:
            # do something
            tool_calls = first_response.choices[0].message.tool_calls
            print(tool_calls)
            messages.append(first_response.choices[0].message)

            for tool in tool_calls:
                tool_name = tool.function.name
                tool_args = json.loads(tool.function.arguments)
                func = tool_map[tool_name]
                tool_res = func(**tool_args)
                messages.append({"tool_call_id":tool.id,"role":"tool","name":tool_name,
                                "content":json.dumps(tool_res)})
        else:
            break
    return first_response.choices[0].message.content

In [35]:
resp = generate_response("what is quantum computing")
print(resp)

[ChatCompletionMessageToolCall(id='call_ESvGskCJ8742yvcgBmaLGFaH', function=Function(arguments='{"query":"Quantum computing"}', name='get_wikipedia_summary'), type='function')]
Quantum computing is a type of computing that uses quantum mechanical phenomena. Unlike classical bits, quantum bits or qubits can exist in multiple states simultaneously, allowing quantum computers to perform certain calculations exponentially faster than classical computers. They have the potential to break encryption and aid in physical simulations, but are currently experimental and not practical for real-world use due to challenges like quantum decoherence. Quantum computers offer time complexity advantages, potentially solving problems that classical computers cannot solve efficiently.


In [36]:
resp = generate_response("what is the current weather in bangalore")
print(resp)

[ChatCompletionMessageToolCall(id='call_sHkAwH0DubfxqhUabA7JNW6V', function=Function(arguments='{"city":"bangalore"}', name='get_current_weather_info'), type='function')]
The current weather in Bangalore is clear sky with a temperature of approximately 31.87°C.


In [37]:
resp = generate_response("what is the current weather in bangalore and london")
print(resp)

[ChatCompletionMessageToolCall(id='call_OofULLem2avk3amWsfrg6159', function=Function(arguments='{"city": "Bangalore"}', name='get_current_weather_info'), type='function'), ChatCompletionMessageToolCall(id='call_NOlRTvEeNaRzQ5DES6zINqXo', function=Function(arguments='{"city": "London"}', name='get_current_weather_info'), type='function')]
The current weather in Bangalore is clear sky with a temperature of about 31.87°C. In London, it is scattered clouds with a temperature of about 17.14°C.


In [38]:
resp = generate_response("what is the current weather in bangalore and tell me which state Bangalore is in")
print(resp)

[ChatCompletionMessageToolCall(id='call_PsVBfR8Tenk0LBTknFLKoXSV', function=Function(arguments='{"city": "Bangalore"}', name='get_current_weather_info'), type='function'), ChatCompletionMessageToolCall(id='call_dRcDmsUdzK9DQjL0DfGBHkgv', function=Function(arguments='{"query": "Bangalore"}', name='get_wikipedia_summary'), type='function')]
The current weather in Bangalore is clear sky with a temperature of approximately 31.87°C (305.02 K). Bangalore is in the state of Karnataka, India.


In [41]:
resp = generate_response("what is the current weather in the capital city of Chhatisgrah.")
print(resp)

[ChatCompletionMessageToolCall(id='call_g5RDjS3MwxBRhOaqwIdggpMn', function=Function(arguments='{"query":"capital city of Chhattisgarh"}', name='get_wikipedia_summary'), type='function')]
[ChatCompletionMessageToolCall(id='call_IqEDYAvgylscpiuWZPhNM8YM', function=Function(arguments='{"city":"Nava Raipur"}', name='get_current_weather_info'), type='function')]


KeyError: 'weather'